*AI-Powered Nutrition Analyzer For Fitness Enthusiasts*
**Team ID PNT2022TMID43392** **--SPRINT1--**

**Importing the libraries**

In [4]:
!pip install keras
!pip install tensorflow

In [ ]:
import numpy as np
import tensorflow
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout 
from tensorflow.keras import layers 
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Sequential 

**Importing Flask libraries**

In [ ]:
!pip install flask

In [ ]:
from flask import Flask, render_template,request
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import requests


**Configure ImageDataGenerator class- Image Data Augmentation**

In [ ]:
train = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test=ImageDataGenerator(rescale=1./255)

In [ ]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='0imlvG2AYR6ttRFokvrlhzHuNScVGPFkbWNx3EyWdeyD',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'ainutritionanalyser-donotdelete-pr-jodnvhxqdlf9gy'
object_key = 'data.zip'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [ ]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [ ]:
pwd

In [ ]:
filenames = os.listdir('/home/wsuser/work/Dataset/TRAIN_SET')

**Apply Image DataGenerator Functionality To Trainset And Testset**
**Loading the data and performing data augmentation**


In [ ]:
x_train = train.flow_from_directory('/home/wsuser/work/Dataset/TRAIN_SET', target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse')
x_test = test.flow_from_directory('/home/wsuser/work/Dataset/TEST_SET', target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse') 

In [ ]:
print(x_train.class_indices)

In [ ]:
print(x_test.class_indices)

In [ ]:
from collections import Counter as c
c(x_train .labels)

**CREATING THE MODEL-->**
**Adding CNN layers**

In [ ]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))

# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())



**Adding Dense Layers**

In [ ]:
# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=5, activation='softmax')) # softmax for more than 2

In [ ]:
classifier.summary()

**Compiling the model**

In [ ]:
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) 

**Fitting the model-->TRAIN THE MODEL**

In [ ]:
classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=20, validation_data=x_test,validation_steps = len(x_test))

**SAVE THE MODEL**

In [ ]:
classifier.save('nutrition.h5')

In [ ]:
!tar -zcvf Nutrition-analyser-model.tgz nutrition.h5

In [ ]:
ls -1

In [ ]:
pip install watson-machine-learning-client --upgrade

In [ ]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey":"SeexqLh7tYnpqmWvUgLHL43ZR8TCqOwYE8Nwl13gNIv3"
}
client = APIClient(wml_credentials)


In [ ]:
client = APIClient(wml_credentials)

In [ ]:
def guid_from_space_name(client,space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [ ]:
space_uid = guid_from_space_name(client, 'Nutrition_analyser')
print("Space UID= " + space_uid)

In [ ]:
client.set.default_space(space_uid)

In [ ]:
client.software_specifications.list()

In [ ]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

In [ ]:
model_details = client.repository.store_model(model='Nutrition-analyser-model.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid}
                                             )
model_id = client.repository.get_model_uid(model_details)

In [ ]:
model_id

In [ ]:
client.repository.download(model_id,'my_model2.tar.gz')

In [ ]:
from keras.models import load_model
from keras.preprocessing import image

In [ ]:
model = load_model("nutrition.h5")

In [ ]:
from PIL import Image

In [ ]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='0imlvG2AYR6ttRFokvrlhzHuNScVGPFkbWNx3EyWdeyD',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'ainutritionanalyser-donotdelete-pr-jodnvhxqdlf9gy'
object_key = '1_100.jpg'

streaming_body_3 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [ ]:
img = image.load_img(streaming_body_3,target_size=(64,64))


In [108]:
pwd

'/home/wsuser/work'

In [109]:
model=load_model('/home/wsuser/work/nutrition.h5')

In [61]:
img=image.load_img(r'/home/wsuser/work/Dataset/TEST_SET/PINEAPPLE/1_100.jpg',target_size=(64,64))

In [111]:
x = image.img_to_array(img)
x = np.expand_dims(x,axis = 0)
pred=model.predict(x) 
y_pred=np.argmax(pred)
y_pred

3

In [112]:
index=['APPLES', 'BANANA', 'ORANGE','PINEAPPLE','WATERMELON']
result = str(index[y_pred])
result

'PINEAPPLE'